In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import pandas as pd

import torch
from Models import *
from functions import *

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [2]:
num_columns = len(np.genfromtxt('./generated_data/pivot_PM10.csv', delimiter=',', max_rows=1))
columns_to_read_PM = [i for i in range(num_columns) if i != 0]

num_columns = len(np.genfromtxt('./generated_data/pivot_traffic.csv', delimiter=',', max_rows=1))
columns_to_read_traffic = [i for i in range(num_columns) if i != 0]

In [3]:
x = np.genfromtxt('./generated_data/pivot_traffic.csv', delimiter=',', skip_header=1, usecols=columns_to_read_traffic, dtype=float)
y = np.genfromtxt('./generated_data/pivot_PM10.csv', delimiter=',', skip_header=1, usecols=columns_to_read_PM,  dtype=float)

In [4]:
print(x.shape)
print(y.shape)
print(x[:,0].shape)
print(y[:,0].shape)

(3983, 82)
(3983, 12)
(3983,)
(3983,)


In [5]:
# import xgboost as xgb

# model = xgb.XGBRegressor()
 
# model.fit(x, y)

# importances = model.feature_importances_
# print(importances)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=True)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
input_size = x.shape[1]
hidden_size = 128
output_size = y.shape[1]
print(input_size)
print(output_size)

# model = MLP(input_size, hidden_size, output_size)
model = MLP_v2(input_size, hidden_size, output_size).to(device)

82
12


In [9]:
#model test
mean_abs_error, _, _ = model_test(model, test_loader, device)
print(f'Mean Absolute Error: {mean_abs_error:.4f}')

Mean Absolute Error: 31.3614


In [10]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [11]:
num_epochs = 700
mean_absolute_error_list = []

for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch) % 50 == 0:
        mean_abs_error, _ , _ = model_test(model, test_loader, device)
        mean_absolute_error_list.append(mean_abs_error)
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}, Mean Absolute Error: {mean_abs_error:.4f}')
        # print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [0/700], Loss: 325.4148, Mean Absolute Error: 15.6349
Epoch [50/700], Loss: 270.6408, Mean Absolute Error: 14.2386
Epoch [100/700], Loss: 190.2000, Mean Absolute Error: 11.0967
Epoch [150/700], Loss: 95.2835, Mean Absolute Error: 7.6582
Epoch [200/700], Loss: 26.8414, Mean Absolute Error: 5.5092
Epoch [250/700], Loss: 16.6017, Mean Absolute Error: 4.9416
Epoch [300/700], Loss: 14.3275, Mean Absolute Error: 4.8516
Epoch [350/700], Loss: 10.6804, Mean Absolute Error: 4.8434
Epoch [400/700], Loss: 10.5453, Mean Absolute Error: 4.8898
Epoch [450/700], Loss: 10.7771, Mean Absolute Error: 4.8980
Epoch [500/700], Loss: 12.4897, Mean Absolute Error: 4.9019
Epoch [550/700], Loss: 11.7797, Mean Absolute Error: 4.9073
Epoch [600/700], Loss: 9.5968, Mean Absolute Error: 4.8835
Epoch [650/700], Loss: 8.9081, Mean Absolute Error: 4.9281


In [12]:
#SAVING TO ONNX
#MPL model
# import torch.onnx
# model.eval()
# batch_size = 512
# x = torch.randn(batch_size,x.shape[1]).to(device)

# output_path = './PM10_model_v1.onnx'

# # Eksport modelu ONNX z dynamicznym rozmiarem wsadu
# torch.onnx.export(model, x, output_path, export_params=True, opset_version=10, do_constant_folding=True,
#                     input_names=['input_0'], output_names=['output_0'],
#                   dynamic_axes={'input_0': {0: 'batch_size'},'output_0': {0: 'batch_size'}},
#                   verbose=True)

In [13]:
mean_abs_error, preds_list, outs_list = model_test(model, test_loader, device)
print(preds_list.shape)

(797, 12)


In [14]:
#calculate the mean absolute error for every output
from sklearn.metrics import mean_absolute_error

mean_abs_error_list = []
for i in range(output_size):
    mean_abs_error = mean_absolute_error(outs_list[:,i], preds_list[:,i])
    mean_abs_error_list.append(mean_abs_error)

print(mean_abs_error_list)

[4.3382344, 3.8167715, 4.349241, 4.3798447, 4.1162553, 4.835694, 5.351705, 5.7931514, 4.890324, 5.585985, 5.1618347, 5.291212]
